In [215]:
import pandas as pd
import numpy as np
import os
#os.chdir("../")
from question_utils import make_db_tables_from_df
import json

In [216]:
data = pd.read_parquet(r'outputs/wucziapping_quest.parquet')

### question category

In [217]:
df1, df2, df3, df4 = make_db_tables_from_df(data)

In [218]:
correct_ans = df1['correct_answer'].to_frame().reset_index(drop=True)
correct_ans

,correct_answer
0,"[paleogen, neogen]"
1,"[paleogen, neogen]"
2,"[paleogen, neogen]"
3,[paleogen]
4,[czwartorzęd]
...,...
5210,"[riak, orosir, stater]"
5211,"[sider, stater, orosir]"
5212,[stater]
5213,"[stater, orosir, sider]"


In [219]:
correct_ans['uni'] = correct_ans['correct_answer'].str.join('')
correct_ans

,correct_answer,uni
0,"[paleogen, neogen]",paleogenneogen
1,"[paleogen, neogen]",paleogenneogen
2,"[paleogen, neogen]",paleogenneogen
3,[paleogen],paleogen
4,[czwartorzęd],czwartorzęd
...,...,...
5210,"[riak, orosir, stater]",riakorosirstater
5211,"[sider, stater, orosir]",siderstaterorosir
5212,[stater],stater
5213,"[stater, orosir, sider]",staterorosirsider


In [220]:
uni_df = correct_ans['uni'].to_frame().drop_duplicates().reset_index()
uni_df

,index,uni
0,0,paleogenneogen
1,3,paleogen
2,4,czwartorzęd
3,6,neogen
4,7,czwartorzędneogen
...,...,...
1904,5208,sider
1905,5209,stater
1906,5211,siderstaterorosir
1907,5213,staterorosirsider


In [221]:
uni_idx = uni_df['index'].values
uni_idx

array([   0,    3,    4, ..., 5211, 5213, 5214], dtype=int64)

In [222]:
correct_ans=correct_ans.reset_index()
correct_ans = correct_ans[correct_ans['index'].isin(uni_idx)]
correct_ans['index']=np.arange(1, len(correct_ans)+1)

In [223]:
correct_ans.rename(columns={'index':'correct_answer_key'}, inplace=True)

In [224]:
correct_ans

,correct_answer_key,correct_answer,uni
0,1,"[paleogen, neogen]",paleogenneogen
3,2,[paleogen],paleogen
4,3,[czwartorzęd],czwartorzęd
6,4,[neogen],neogen
7,5,"[czwartorzęd, neogen]",czwartorzędneogen
...,...,...,...
5208,1905,[sider],sider
5209,1906,[stater],stater
5211,1907,"[sider, stater, orosir]",siderstaterorosir
5213,1908,"[stater, orosir, sider]",staterorosirsider


In [225]:
df1['corr_ans_key'] = df1['correct_answer'].str.join('')

In [226]:
df1=pd.merge(
    df1,
    correct_ans[['correct_answer_key','uni']],
    how='left',
    left_on='corr_ans_key',
    right_on='uni'
).drop(columns=['correct_answer','corr_ans_key','uni'])

In [230]:
correct_ans = correct_ans[['correct_answer_key','correct_answer']]

In [232]:
correct_ans=correct_ans.explode('correct_answer')

In [233]:
correct_ans

,correct_answer_key,correct_answer
0,1,paleogen
0,1,neogen
3,2,paleogen
4,3,czwartorzęd
6,4,neogen
...,...,...
5213,1908,orosir
5213,1908,sider
5214,1909,orosir
5214,1909,stater


In [234]:
df1

,category_key,possible_answers_key,question_txt,correct_answer_key
0,1,1,1,1
1,1,2,1,1
2,1,3,1,1
3,1,4,1,2
4,1,5,1,3
...,...,...,...,...
5210,1,5211,61,1898
5211,1,5212,61,1907
5212,1,5213,61,1906
5213,1,5214,61,1908


In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

### NIE-PREKAMBR - SYSTEM ERY - single - easy

In [54]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)

In [60]:
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [77]:
question_text = []
possible_answers = []
correct_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    temp_possible_answers = np.concatenate(
        (np.random.choice(wrong_array, 3, replace=False), np.random.choice(correct_array, 1))
    )
    np.random.shuffle(temp_possible_answers)
    temp_possible_answers = np.append(temp_possible_answers, np.nan)
    for correct in temp_possible_answers:
        if correct in correct_array:
            correct_answers.append(correct)
            continue

    possible_answers.append(temp_possible_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1).drop_duplicates()

### MULTI

In [21]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [53]:
question_text = []
wrong_answers = []
correct_answers = []
possible_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    if len(correct_array)>5:
        randomSampleCorrectSize = np.random.randint(1, 6)
    else:
        randomSampleCorrectSize = np.random.randint(1,len(correct_array))

    wrongSample = 5-randomSampleCorrectSize

    temp_wrong_answers = np.random.choice(wrong_array, wrongSample, replace=False)
    temp_correct_answers = np.random.choice(correct_array, randomSampleCorrectSize, replace=False)
    temp_possible_answers = np.concatenate((temp_correct_answers, temp_wrong_answers))
    np.random.shuffle(temp_possible_answers)
    possible_answers.append(temp_possible_answers)

    correct_answers.append(temp_correct_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1)

In [3]:
from question_builder import QuestionBuilder

no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

test = QuestionBuilder(no_precambr_data, precambr_data)

In [4]:
dfs = []
criteria = [
    (False, True, True, 1, 9, "ERA", "SYSTEM", "Wybierz system ery", []),
    (False, True, False, 1, 9, "ERA", "PIETRO", "Wybierz piętro ery", []),
    (False, True, False, 1, 9, "SYSTEM", "ODDZIAL", "Wybierz system oddziału", ['kreda','jura','trias','dewon','ordowik']),
    (False, True, False, 1, 9, "SYSTEM", "PIETRO", "Wybierz piętro systemu", []),
    (False, True, False, 1, 9, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału", ['przidoli']),
    # prek
    (True, True, True, 1, 9, "ERA", "SYSTEM", "Wybierz system ery", ['neoarchaik','mezoarchaik','paleoarchaik','eoarchaik','brak']),
]

n = 0
for is_prek, if_multi, both_scopes, multiplyIdx, possAns, targetName, scopeName, questionTxt, explusionList in criteria:
    temp = test.prepare_closed_ended(is_prek, if_multi, both_scopes, multiplyIdx, possAns, targetName, scopeName, questionTxt, explusionList)
    dfs.append(temp)
    n+=1
    print(n)

1
2
3
4
5
6


In [11]:
all_dfs= pd.concat(dfs, ignore_index=True)

In [13]:
all_dfs.to_excel('outputs/wucziapping_quest.xlsx',index=False)

In [14]:
all_dfs

,question,correct_answer,A,B,C,D,E,F,G,H,I
0,Wybierz system ery kenozoik,"[paleogen, neogen]",dewon,paleogen,trias,ordowik,karbon,kriogen,neogen,sylur,sider
1,Wybierz system ery kenozoik,"[paleogen, neogen]",dewon,riak,sider,paleogen,kambr,trias,ektas,ordowik,neogen
2,Wybierz system ery kenozoik,"[paleogen, neogen]",kreda,ediakar,neogen,trias,kalim,karbon,sylur,paleogen,stater
3,Wybierz system ery kenozoik,[paleogen],sylur,stater,sten,kreda,kalim,dewon,ektas,sider,paleogen
4,Wybierz system ery kenozoik,[czwartorzęd],sylur,karbon,orosir,ektas,stater,riak,sten,czwartorzęd,trias
...,...,...,...,...,...,...,...,...,...,...,...
5210,Wybierz system ery paleoproterozoik,"[riak, orosir, stater]",trias,orosir,kriogen,ordowik,czwartorzęd,sylur,riak,paleogen,stater
5211,Wybierz system ery paleoproterozoik,"[sider, stater, orosir]",sider,stater,trias,ordowik,neogen,orosir,sten,perm,kambr
5212,Wybierz system ery paleoproterozoik,[stater],ediakar,ton,kriogen,ordowik,perm,dewon,jura,sten,stater
5213,Wybierz system ery paleoproterozoik,"[stater, orosir, sider]",neogen,czwartorzęd,ordowik,sider,orosir,perm,jura,stater,karbon


In [4]:
dfs = []
criteria = [
    # NON PRECAMBRIAN
    # era
    (False, False, 3, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, True, 15, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, False, 3, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, True, 15, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, False, 3, "ERA", "PIETRO", "Wybierz piętro ery"),
    (False, True, 15, "ERA", "PIETRO", "Wybierz piętro ery"),
    # system
    (False, False, 3, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, True, 15, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, False, 3, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    (False, True, 15, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    # oddział
    (False, False, 3, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),
    (False, True, 15, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),

    ## PRECAMBRIAN
    # prekambr
    #(True, False, 2, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, False, 2, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, False, 2, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    (True, True, 10, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    # eon
    (True, False, 2, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, True, 10, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, False, 2, "Eon", "System", "Prekambr - Wybierz system enou"),
    (True, True, 10, "Eon", "System", "Prekambr - Wybierz system enou"),
    # system ery
    (True, False, 2, "Era", "System", "Prekambr - Wybierz system ery"),
    (True, True, 10, "Era", "System", "Prekambr - Wybierz system ery"),

]
n = 0
for is_prek, is_hard, coeff, key1, key2, quest_txt in criteria:
    temp = test.prepare_single(is_prek, is_hard, coeff, key1, key2, quest_txt)
    dfs.append(temp)
    n+=1
    print(n)
    print(f"{is_prek} {is_hard} {coeff} {key1}, {key2}, {quest_txt}")

12
1
False False 3 ERA, SYSTEM, Wybierz system ery
172
2
False True 15 ERA, SYSTEM, Wybierz system ery
34
3
False False 3 ERA, ODDZIAL, Wybierz oddział ery
172
4
False True 15 ERA, ODDZIAL, Wybierz oddział ery
102
5
False False 3 ERA, PIETRO, Wybierz piętro ery
172
6
False True 15 ERA, PIETRO, Wybierz piętro ery
34
7
False False 3 SYSTEM, ODDZIAL, Wybierz oddział systemu
172
8
False True 15 SYSTEM, ODDZIAL, Wybierz oddział systemu
102
9
False False 3 SYSTEM, PIETRO, Wybierz piętro systemu
172
10
False True 15 SYSTEM, PIETRO, Wybierz piętro systemu
102
11
False False 3 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
12
False True 15 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
13
True True 10 Jednostka nieformalna, Eon, Prekambr - Wybierz eon prekambru
8
WARNING!!!!!-------------------
[]
3


ValueError: 

In [4]:
c = (True, True, 5, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru")
df1 = test.prepare_single(c[0], c[1], c[2], c[3], c[4], c[5])

['aalen' 'aeron' 'akwitan' 'alb' 'anizyk' 'apt' 'archaik' 'artyńsk'
 'assel' 'bajos' 'barrem' 'barton' 'baszkir' 'baton' 'berrias' 'brak'
 'burdygał' 'cenoman' 'chiban' 'cisural' 'czangsing' 'czwartorzęd' 'dan'
 'daping' 'darriwil' 'dewon' 'dewon dolny' 'dewon górny' 'dewon środkowy'
 'drum' 'dżangszan' 'ediakar' 'eifel' 'ektas' 'ems' 'eoarchaik' 'eocen'
 'epoka 2' 'famen' 'floj' 'fortun' 'fran' 'furong' 'gelas' 'gorst'
 'grenland' 'gużang' 'gwadelup' 'gżel' 'hadeik' 'hettang' 'hirnant'
 'holocen' 'homer' 'hoteryw' 'ind' 'iprez' 'jura' 'jura dolna'
 'jura górna' 'jura środkowa' 'kalabr' 'kalim' 'kambr' 'kampan' 'kapitan'
 'karbon' 'karnik' 'kasim' 'kat' 'kelowej' 'kenozoik' 'kimeryd' 'koniak'
 'kreda' 'kreda dolna' 'kreda górna' 'kriogen' 'kungur' 'ladyn' 'landower'
 'lang' 'lochkow' 'loping' 'ludford' 'ludlow' 'lutet' 'mastrycht'
 'megalaj' 'messyn' 'mezoarchaik' 'mezoproterozoik' 'mezozoik' 'miaoling'
 'miocen' 'mississip' 'moskow' 'neoarchaik' 'neogen' 'neoproterozoik'
 'northgrip' 

In [5]:
# df_easy = test.prepare_single(False, False, 3, "ERA", "SYSTEM", "Wybierz system ery")
# df_hard = test.prepare_single(False, True, 5, "ERA", "SYSTEM", "Wybierz system ery")

,question,correct_answer,A,B,C,D,E
0,Wybierz system ery kenozoik,neogen,trias,neogen,kreda,perm,NaN
1,Wybierz system ery kenozoik,czwartorzęd,dewon,czwartorzęd,kambr,jura,NaN
2,Wybierz system ery kenozoik,czwartorzęd,dewon,karbon,czwartorzęd,kambr,NaN
3,Wybierz system ery kenozoik,neogen,jura,neogen,sylur,kambr,NaN
4,Wybierz system ery kenozoik,czwartorzęd,dewon,perm,czwartorzęd,kreda,NaN
...,...,...,...,...,...,...,...
103,Wybierz system ery paleozoik,ordowik,jura,czwartorzęd,ordowik,neogen,NaN
104,Wybierz system ery paleozoik,sylur,czwartorzęd,paleogen,jura,sylur,NaN
105,Wybierz system ery paleozoik,kambr,kreda,trias,paleogen,kambr,NaN
106,Wybierz system ery paleozoik,ordowik,neogen,ordowik,jura,czwartorzęd,NaN


In [3]:
import pandas as pd
import numpy as np
import os

In [20]:
non_prec_single = pd.read_csv(fr"{os.getcwd()}\outputs\non_precambrian_questions_single.csv")
non_prec_multi = pd.read_csv(fr"{os.getcwd()}\outputs\non_precambrian_questions_multi.csv")
prec_single = pd.read_csv(fr"{os.getcwd()}\outputs\precambrian_questions_single.csv")
prec_multi = pd.read_csv(fr"{os.getcwd()}\outputs\precambrian_questions_multi.csv")

In [26]:
dfs = [
    non_prec_single,
    non_prec_multi,
    prec_single,
    prec_multi
]

In [31]:
n=0
new_dfs = []
for df in dfs:
    temp = df.copy()
    temp = temp.drop_duplicates()
    temp['isHard'] = np.where(pd.isnull(temp['E']), 0, 1)

    if n%2==0:
        temp['isMulti'] = 0
    else:
        temp['isMulti'] = 1

    temp = temp.reset_index()
    new_dfs.append(temp)
    n+=1

In [35]:
df_final = pd.concat(new_dfs)

In [38]:
df_final.to_csv(fr"{os.getcwd()}\outputs\all_questions_main_df.csv")